# Code up class to perform different tasks

In [1]:
# interactive figures
%matplotlib widget 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# model evalution metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# data preprocessing
from sklearn.preprocessing import normalize

# predictive models
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor


In [3]:
class time_series_prediction():

    def __init__(self,one_d_time_series,lag_window_length,n_ahead_prediction):

        # raw input data + settings for time series -> supervised learning ML problem
        self.one_d_time_series = np.array(one_d_time_series)      # time series array, to array ensure index works as expected for class methods
        self.lag_window_length = lag_window_length                # length of lag window
        self.n_ahead_prediction = n_ahead_prediction              # time ahead to predict

        # transfromed data: set after calling .sliding_window_1()
        self.input_data = None
        self.target_data = None

        # testing and training data: set after calling .train_test_split()
        self.training_split = None
        self.X_test = None
        self.X_train = None
        self.y_test = None
        self.y_train = None

# ****************************************************************************************************************
    # data wrangling
# ****************************************************************************************************************

    # method to transfroms 1-D time series to supervised ML problem: one step ahead forecasting   
    def sliding_window_1(self,verbose):
        # initialize input array
        num_rows = len(self.one_d_time_series) - self.lag_window_length
        array = np.zeros((num_rows, self.lag_window_length + 1))
        
        # loop through data and populate array
        for i in range(num_rows):
            # input features
            array[i,0:self.lag_window_length+1] = self.one_d_time_series[i:i+self.lag_window_length+1]
            # target feature/s
            array[i,-1] = self.one_d_time_series[i+self.lag_window_length]
            
            if verbose == 1:
                # show pattern
                print(array[i,0:self.lag_window_length],' : ',array[i,self.lag_window_length])

        # save results as a class attribute
        self.input_data = array[:,0:self.lag_window_length]
        self.target_data = array[:,self.lag_window_length]

    # method to perform a training and testing split for dataset with only a single column of target variables
    def train_test_split(self,split):
        self.training_split = split
        self.X_train = self.input_data[0:split,:]
        self.X_test = self.input_data[split:,:]
        self.y_train = self.target_data[0:split]
        self.y_test = self.target_data[split:]

    # method to plot testing and training split of data
    def test_train_plot(self):
        num_days = range(len(self.one_d_time_series)) # hacking this as x-axis of plot
        fig, ax = plt.subplots(figsize=(10,5))

        ax.plot(num_days[0:self.training_split] ,self.one_d_time_series[0:self.training_split],'k-',label='Training data') # replace returns with sp_500 for other data plotting
        ax.plot(num_days[self.training_split:] ,self.one_d_time_series[self.training_split:],'r-',label='Testing data')
        plt.plot(num_days[self.training_split+self.lag_window_length:] ,self.y_test,'o',label='Windowed testing data') # important to match time by start 5 (length of time window) after where segmented our testing and training data
        plt.legend(loc=0)  

# ****************************************************************************************************************
    # predictive models
# ****************************************************************************************************************

    def linear_regression(self):
        print('Training multivariate linear regression:')
        # train model
        reg_model = LinearRegression().fit(self.X_train,self.y_train)
        print('Linear regression coefficients: \n',reg_model.coef_)

        # test model
        predictions = reg_model.predict(self.X_test)

        # evaluate: use sklearn metric methods to calc rmse and mae
        mse = mean_squared_error(y_test,predictions)
        mae = mean_absolute_error(y_test,predictions)

        print('RMSE: ',np.sqrt(mse))
        print('MAE: ',mae)

    def support_vector_machine(self):
        print('Training support vector machine:')
        # train model
        svm_regres = LinearSVR(max_iter=1000).fit(X_train,y_train)

        # predict
        svm_predictions = svm_regres.predict(X_test)

        # evaluate
        mse = mean_squared_error(y_test,svm_predictions[:])
        mae = mean_absolute_error(y_test,svm_predictions[:])

        print('RMSE: ',np.sqrt(mse))
        print('MAE: ',mae)

    def neural_net_mlp(self):
        print('Training neural network: ')
        # train neural network
        nn_regres = MLPRegressor(hidden_layer_sizes=(100,100,100),shuffle=False,random_state=1, 
                                max_iter=1000,verbose=1).fit(X_train,y_train)

        # make predictions
        nn_predictions = nn_regres.predict(X_test)

        # evaluate
        mse = mean_squared_error(y_test,nn_predictions[:])
        mae = mean_absolute_error(y_test,nn_predictions[:])

        print('RMSE: ',np.sqrt(mse))
        print('MAE: ',mae)

In [4]:
# import some data
sp_500 = pd.read_csv('../test_data/GSPC.csv')
sp_500

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...,...
17213,2018-05-31,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000
17214,2018-06-01,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000
17215,2018-06-04,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000
17216,2018-06-05,2748.459961,2752.610107,2739.510010,2748.800049,2748.800049,3517790000


In [5]:
normal = time_series_prediction(sp_500['Volume'][-2000:],5,1)
normal.sliding_window_1(verbose=0)
normal.train_test_split(split=1200)

In [6]:
normal.one_d_time_series

array([3896410000, 6136700000, 5067080000, ..., 3376510000, 3517790000,
       3651640000], dtype=int64)

In [7]:
normal.input_data.shape

(1995, 5)

In [8]:
normal.target_data.shape

(1995,)

In [9]:
normal.test_train_plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …